<a href="https://colab.research.google.com/github/ASHIKalip/DataScience/blob/main/LeNet_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# If TensorFlow and TensorFlow Model Optimization Toolkit are not installed, use the following to install them
!pip install tensorflow
!pip install tensorflow-model-optimization



In [17]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Add a channel dimension to the data (MNIST is grayscale, so add a single channel)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Define a simple LeNet model
def create_lenet_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(6, (5, 5), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(16, (5, 5), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation='relu'),
        tf.keras.layers.Dense(84, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # Output layer for 10 classes
    ])
    return model

# Create and compile the model
model = create_lenet_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)


Epoch 1/5
1875/1875 [==============================] - 50s 25ms/step - loss: 0.2142 - accuracy: 0.9328 - val_loss: 0.0651 - val_accuracy: 0.9799
Epoch 2/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0705 - accuracy: 0.9775 - val_loss: 0.0642 - val_accuracy: 0.9792
Epoch 3/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0503 - accuracy: 0.9847 - val_loss: 0.0390 - val_accuracy: 0.9874
Epoch 4/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0394 - accuracy: 0.9875 - val_loss: 0.0345 - val_accuracy: 0.9887
Epoch 5/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0319 - accuracy: 0.9900 - val_loss: 0.0337 - val_accuracy: 0.9887


In [18]:
import tensorflow as tf

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('lenet_mnist_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [19]:
# Apply pruning to the LeNet model
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.5,
        final_sparsity=0.8,
        begin_step=0,
        end_step=1000  # Adjust based on training steps
    )
}

# Prune the model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Add the UpdatePruningStep callback
pruning_callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='log_dir'),  # Optional: For logging and visualization
]

# Re-compile the pruned model
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the pruned model with the callback
pruned_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, callbacks=pruning_callbacks)
#pruned_model.save('pruned_lenet_mnist.h5')

# Strip pruning wrappers (necessary before quantization)
stripped_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Apply quantization aware training (QAT)
qat_model = tfmot.quantization.keras.quantize_model(stripped_model)

# Re-compile the quantized model
qat_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train with quantization aware training
qat_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3)

# Save the quantized model
qat_model.save('quantized_lenet_mnist.h5')

# Evaluate the quantized model on the test data
qat_model.evaluate(x_test, y_test)

Epoch 1/3
   5/1875 [..............................] - ETA: 30s - loss: 0.0110 - accuracy: 0.9875    

1875/1875 [==============================] - 40s 18ms/step - loss: 0.0495 - accuracy: 0.9852 - val_loss: 0.0396 - val_accuracy: 0.9874
Epoch 2/3
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0298 - accuracy: 0.9905 - val_loss: 0.0310 - val_accuracy: 0.9896
Epoch 3/3
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0236 - accuracy: 0.9926 - val_loss: 0.0307 - val_accuracy: 0.9898
Epoch 1/3
1875/1875 [==============================] - 46s 22ms/step - loss: 0.0377 - accuracy: 0.9882 - val_loss: 0.0409 - val_accuracy: 0.9869
Epoch 2/3
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0259 - accuracy: 0.9920 - val_loss: 0.0330 - val_accuracy: 0.9888
Epoch 3/3
  1/313 [..............................] - ETA: 9s - loss: 0.0212 - accuracy: 1.0000

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 2s 8ms/step - loss: 0.0292 - accuracy: 0.9918


[0.029169641435146332, 0.9918000102043152]

In [20]:
import tensorflow as tf

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('lenet_mnist_stripped.tflite', 'wb') as f:
    f.write(tflite_model)

In [21]:
pruned_model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0307 - accuracy: 0.9898


[0.030690917745232582, 0.989799976348877]

In [22]:
pruned_model.save('pruned_lenet_mnist.h5')


/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
import tensorflow as tf

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('lenet_mnist_pruned.tflite', 'wb') as f:
    f.write(tflite_model)

In [24]:
pruned_model.save('pruned_lenet_mnist.keras')

In [25]:
qat_model.save('quantized_lenet_mnist.keras')

In [26]:
model.save('model_lenet_mnist.keras')
model.save('model_lenet_mnist.h5')

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
import tensorflow as tf

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('lenet_mnist_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

In [28]:
import h5py
import heapq
import collections
import numpy as np

# Define a Huffman node class
class HuffmanNode:
    def __init__(self, frequency, symbol=None, left=None, right=None):
        self.frequency = frequency
        self.symbol = symbol
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.frequency < other.frequency

# Create Huffman tree from weight frequencies
def create_huffman_tree(frequency):
    heap = []
    for symbol, freq in frequency.items():
        heapq.heappush(heap, HuffmanNode(freq, symbol))

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        new_node = HuffmanNode(left.frequency + right.frequency, None, left, right)
        heapq.heappush(heap, new_node)

    return heapq.heappop(heap)

# Generate Huffman codes by traversing the Huffman tree
def create_huffman_codes(root, code='', code_dict=None):
    if code_dict is None:
        code_dict = {}
    if root.symbol is not None:
        code_dict[root.symbol] = code
    else:
        create_huffman_codes(root.left, code + '0', code_dict)
        create_huffman_codes(root.right, code + '1', code_dict)
    return code_dict

# Function to calculate frequencies of unique weight values
def calculate_weight_frequencies(weights):
    unique, counts = np.unique(weights, return_counts=True)
    frequency = dict(zip(unique, counts))
    return frequency

# Huffman coding for Keras model
model_file = 'quantized_lenet_mnist.h5'

with h5py.File(model_file, 'r') as f:
    # Traverse the HDF5 structure to find weights
    for group_name, group in f.items():
        if isinstance(group, h5py.Group):
            for dataset_name in group.keys():
                dataset = group[dataset_name]
                if isinstance(dataset, h5py.Dataset):
                    try:
                        # Access the dataset values
                        if 'kernel:0' in dataset_name or 'bias:0' in dataset_name:
                            weights = dataset[:]
                            frequency = calculate_weight_frequencies(weights)

                            # Create Huffman tree and generate codes
                            huffman_tree = create_huffman_tree(frequency)
                            huffman_codes = create_huffman_codes(huffman_tree)

                            print(f"Huffman Codes for {group_name}/{dataset_name}: {huffman_codes}")
                    except Exception as e:
                        print(f"Error processing {group_name}/{dataset_name}: {e}")


In [29]:
# Read the TFLite model
import h5py
import heapq
import numpy as np
import collections

with open('lenet_mnist_quantized.tflite', 'rb') as f:
    tflite_data = f.read()

# Calculate frequency of bytes
byte_frequencies = collections.Counter(tflite_data)

# Create Huffman tree and generate codes
huffman_tree = create_huffman_tree(byte_frequencies)
huffman_codes = create_huffman_codes(huffman_tree)

print("Huffman Codes for TFLite model:", huffman_codes)

# Apply Huffman codes to compress the TFLite model
compressed_tflite_data = []
for byte in tflite_data:
    compressed_tflite_data.append(huffman_codes[byte])

# Convert the compressed data to binary
compressed_tflite_data = ''.join(compressed_tflite_data)

# Save the compressed TFLite model
with open('compressed_lenet_mnist.tflite', 'wb') as f:
    f.write(bytes(compressed_tflite_data, 'utf-8'))






Huffman Codes for TFLite model: {130: '00000', 190: '00001', 162: '0001000', 111: '00010010', 18: '000100110', 72: '000100111000', 123: '00010011100100', 175: '000100111001010', 119: '000100111001011', 233: '0001001110011', 197: '00010011101', 182: '0001001111', 220: '0001010', 146: '0001011', 93: '00011000000', 205: '00011000001', 224: '0001100001', 142: '000110001', 214: '00011001', 68: '0001101000', 239: '00011010010', 201: '000110100110', 234: '000110100111', 105: '000110101', 16: '0001101100', 63: '0001101101', 86: '0001101110', 40: '00011011110', 7: '00011011111', 34: '000111', 211: '0010000', 23: '00100010', 243: '001000110', 102: '0010001110', 229: '00100011110', 208: '00100011111', 231: '00100100', 139: '001001010', 199: '0010010110', 248: '00100101110', 15: '001001011110', 171: '001001011111', 117: '001001100', 166: '0010011010', 28: '00100110110', 94: '001001101110', 160: '0010011011110', 27: '0010011011111', 49: '00100111', 31: '00101', 189: '0011', 21: '0100', 8: '0101', 5

In [30]:


import tensorflow as tf
import collections
import heapq

# Load the TensorFlow Lite model
with open("lenet_mnist_quantized.tflite", "rb") as f:
    tflite_data = f.read()

# Step 1: Calculate frequency of each byte in the model
byte_frequencies = collections.Counter(tflite_data)

# Step 2: Create a Huffman Tree
class HuffmanNode:
    def __init__(self, byte, frequency):
        self.byte = byte
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        # Compare nodes by frequency
        return self.frequency < other.frequency

def build_huffman_tree(frequencies):
    heap = [HuffmanNode(byte, freq) for byte, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = HuffmanNode(None, left.frequency + right.frequency)
        merged.left = left
        merged.right = right

        heapq.heappush(heap, merged)

    return heap[0]

# Step 3: Generate Huffman codes from the Huffman Tree
def generate_huffman_codes(node, prefix="", code_map=None):
    if code_map is None:
        code_map = {}

    if node.byte is not None:
        # It's a leaf node, add it to the code map
        code_map[node.byte] = prefix
    else:
        # It's an internal node, recurse on children
        generate_huffman_codes(node.left, prefix + "0", code_map)
        generate_huffman_codes(node.right, prefix + "1", code_map)

    return code_map

# Build the Huffman tree and generate the codes
huffman_tree = build_huffman_tree(byte_frequencies)
huffman_codes = generate_huffman_codes(huffman_tree)

# Step 4: Compress the TensorFlow Lite model using Huffman codes
compressed_tflite_data = []
for byte in tflite_data:
    compressed_tflite_data.append(huffman_codes[byte])

# Step 5: Convert the compressed data to a binary representation
compressed_tflite_binary = "".join(compressed_tflite_data)

# Convert the binary string into bytes
compressed_tflite_bytes = int(compressed_tflite_binary, 2).to_bytes(
    (len(compressed_tflite_binary) + 7) // 8, "big"
)

# Step 6: Save the compressed data to a file
with open("compressed_lenet_mnist.tflite", "wb") as f:
    f.write(compressed_tflite_bytes)

print("Huffman compression completed and saved to 'compressed_lenet_mnist.tflite'")


Huffman compression completed and saved to 'compressed_lenet_mnist.tflite'
